# README

This notebook allows to prepare dataset of slices.

Data are taken after creation of a dataset from nii.gz sequences with `0 - Prepare Size and Slices.ipynb`.

Dataset created by this notebook is a pair of dictionaries:
* first dictionary's key is the subject directory, value is the list of normalized slices
* second dictionary's key is the subject directory, value is the label for that subject

This structure will be used in random partitioning and shuffling to setup a set of labels and slices.

Dataset subject list for a given set of sequences is validated: only subjects that own all the sequences are considered: the others are filtered.

In [1]:
from gliomi import *

In [2]:
import numpy as np

In [3]:
class SliceProvider:
    
    def __init__(self, sequence_repo, full_brain=False):
        self.sequence_repo = sequence_repo
        self.full_brain = full_brain
        
    def has(self, subject, sequence_name):
        return self.sequence_repo.has(subject, sequence_name)
        
    """
    Save the slices of the whole brain reshaped with a squared size
    """
    def get_slices(self, sequence_name, subject, side):
        
        print("Loading slices from:", subject)
    
        return get_slices_for_subject(self.sequence_repo, 
                                      sequence_name, 
                                      subject, 
                                      side, 
                                      full_brain=self.full_brain)

In [4]:
import pickle
import glob
import os
import pandas as pd
from numpy import load
import numpy as np
import sys

class GliomiDataset:
    
    def __init__(self, subjects_dir, slice_provider):
        self.slice_provider = slice_provider
        self.subjects_dir = subjects_dir
    
    """
    Load a set of slices given the ROI sizes and the selected percentile for a single subject.
    Only slices with a ROI size greater or equal that corresponding percentile are considered.
    """
    def load_data_single_channel_for_subject(self, subject, sequence_name, side, percentile):

        # Load sizes
        with open(f"/data/RMN/dataset-gliomi-cnn/roi-sizes/roi-sizes-{subject}-{side}.pickle", "rb") as file:
            roi_sizes = pickle.load(file)
    
        # Load slices
        slices = self.slice_provider.get_slices(sequence_name, subject, side)

        print(f"Loaded {subject}: {slices.shape}, {roi_sizes.shape}")

        return slices[:,:,ordered_index_percentile_of_sizes(roi_sizes, percentile)]    
    
    """
    Load a set of slices given the ROI sizes and the selected percentile for a set of subjects.
    Only slices with a ROI size greater or equal that corresponding percentile are considered.
    Images channels correspondes to specified sequences given in the same order.
    """
    def load_data(self, subjects, sequence_names, side, percentile):
        slices = [
            np.stack([
                self.load_data_single_channel_for_subject(subject, sequence_name, side, percentile) 
                for sequence_name in sequence_names], axis=3)
            for subject in subjects
        ]
        return np.moveaxis(np.concatenate(slices, axis=2), 2, 0)
    
    """
    Return a valid list of subjects that has all the sequences specified.
    Subjects are selected between the list of specified
    """
    def get_valid_subject_list(self, subjects, sequence_names):
        ret = []
        for subject in subjects:
            admit = True
            for sequence_name in sequence_names:
                if not self.slice_provider.has(subject, sequence_name):
                    admit = False
            if admit:
                ret.append(subject)
        return ret
    
    """
    Return list of subjects in a dataset dir: all the directory names are returned as subjects
    """
    def get_subject_list(self):
        ret = []
        for file in glob.glob(f"{self.subjects_dir}/*"):
            if os.path.isdir(file):
                ret.append(os.path.basename(file))
        return ret
    

    """
    Given subject path and a table of subjects and labels for a given side
    Create a structure that hosts all related slices
    """
    def load_dataset(self, sequence_names, side, percentile):

        all_subjects = self.get_subject_list()

        # Return only subjects that has all sequences
        valid_subjects = self.get_valid_subject_list(
            all_subjects, 
            sequence_names)

        X = {}

        for subject in valid_subjects:
            images = self.load_data([subject], sequence_names, side, percentile)
            X[subject] = normalize(images, max_value=1., axis=(1, 2))

        return X

# Dataset Management

In [5]:
sequence_repo = SequenceRepoGliomi("/data/RMN/dataset-gliomi")

slice_provider = SliceProvider(sequence_repo, full_brain=True)

dataset = GliomiDataset("/data/RMN/dataset-gliomi/scaled", slice_provider)

In [12]:
side = 224

percentile = 100

for sequence_name in ["T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], side, percentile)
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

0 ALESSANDRINI_GLAUCO
Loading slices from: ALESSANDRINI_GLAUCO
Loaded ALESSANDRINI_GLAUCO: (224, 224, 63), (63,)
1 ASTA_G
Loading slices from: ASTA_G
Loaded ASTA_G: (224, 224, 48), (48,)
2 BAGNOLI_VINCENZO
Loading slices from: BAGNOLI_VINCENZO
Loaded BAGNOLI_VINCENZO: (224, 224, 92), (92,)
3 BALZAMO_A
Loading slices from: BALZAMO_A
Loaded BALZAMO_A: (224, 224, 145), (145,)
4 BARONTINI_MARIA_GIOVANNA
Loading slices from: BARONTINI_MARIA_GIOVANNA
Loaded BARONTINI_MARIA_GIOVANNA: (224, 224, 56), (56,)
5 BATTISTA_DOMENICA
Loading slices from: BATTISTA_DOMENICA
Loaded BATTISTA_DOMENICA: (224, 224, 54), (54,)
6 BERGNACH_SILVANO
Loading slices from: BERGNACH_SILVANO
Loaded BERGNACH_SILVANO: (224, 224, 68), (68,)
7 BERNOLA_TERESA
Loading slices from: BERNOLA_TERESA
Loaded BERNOLA_TERESA: (224, 224, 32), (32,)
8 BERTUZZI_LUISA
Loading slices from: BERTUZZI_LUISA
Loaded BERTUZZI_LUISA: (224, 224, 89), (89,)
9 BEVILACQUA_RITA
Loading slices from: BEVILACQUA_RITA
Loaded BEVILACQUA_RITA: (224, 224,

Loaded NERONE_GIANLUCA: (224, 224, 67), (67,)
83 NERVEGNA_G
Loading slices from: NERVEGNA_G
Loaded NERVEGNA_G: (224, 224, 148), (148,)
84 ORLANDI_PAOLO
Loading slices from: ORLANDI_PAOLO
Loaded ORLANDI_PAOLO: (224, 224, 6), (6,)
85 PAGANNONE_GIANNI
Loading slices from: PAGANNONE_GIANNI
Loaded PAGANNONE_GIANNI: (224, 224, 67), (67,)
86 PAGLIAROLI_LUCIA
Loading slices from: PAGLIAROLI_LUCIA
Loaded PAGLIAROLI_LUCIA: (224, 224, 39), (39,)
87 PAGNOTTA
Loading slices from: PAGNOTTA
Loaded PAGNOTTA: (224, 224, 59), (59,)
88 PALMA
Loading slices from: PALMA
Loaded PALMA: (224, 224, 73), (73,)
89 PALMIERI
Loading slices from: PALMIERI
Loaded PALMIERI: (224, 224, 83), (83,)
90 PANETTI
Loading slices from: PANETTI
Loaded PANETTI: (224, 224, 90), (90,)
91 PASCAL
Loading slices from: PASCAL
Loaded PASCAL: (224, 224, 33), (33,)
92 PASSARI
Loading slices from: PASSARI
Loaded PASSARI: (224, 224, 68), (68,)
93 PELUSO_A
Loading slices from: PELUSO_A
Loaded PELUSO_A: (224, 224, 58), (58,)
94 PENNICCHI_R


Loaded CRESCENZI_ARMANDO: (224, 224, 87), (87,)
33 DARIDA
Loading slices from: DARIDA
Loaded DARIDA: (224, 224, 46), (46,)
34 DEL_BOVE_PIERINA
Loading slices from: DEL_BOVE_PIERINA
Loaded DEL_BOVE_PIERINA: (224, 224, 57), (57,)
35 DE_PAOLI_R
Loading slices from: DE_PAOLI_R
Loaded DE_PAOLI_R: (224, 224, 149), (149,)
36 DE_SANTIS_GIORGO
Loading slices from: DE_SANTIS_GIORGO
Loaded DE_SANTIS_GIORGO: (224, 224, 63), (63,)
37 DIASPRO_G
Loading slices from: DIASPRO_G
Loaded DIASPRO_G: (224, 224, 165), (165,)
38 DI_CARLATONIO_MAURIZIO
Loading slices from: DI_CARLATONIO_MAURIZIO
Loaded DI_CARLATONIO_MAURIZIO: (224, 224, 43), (43,)
39 DI_LORENZO_TOMMASO
Loading slices from: DI_LORENZO_TOMMASO
Loaded DI_LORENZO_TOMMASO: (224, 224, 10), (10,)
40 DI_MARCO_L
Loading slices from: DI_MARCO_L
Loaded DI_MARCO_L: (224, 224, 72), (72,)
41 DI_MASO_SIMONE
Loading slices from: DI_MASO_SIMONE
Loaded DI_MASO_SIMONE: (224, 224, 84), (84,)
42 DI_MASSA_SERGIO
Loading slices from: DI_MASSA_SERGIO
Loaded DI_MASSA_

Loaded RE_ATTILIO: (224, 224, 50), (50,)
117 RICCI_ALESSANDRO
Loading slices from: RICCI_ALESSANDRO
Loaded RICCI_ALESSANDRO: (224, 224, 49), (49,)
118 ROMITO_ORAZIO
Loading slices from: ROMITO_ORAZIO
Loaded ROMITO_ORAZIO: (224, 224, 53), (53,)
119 RONZO_F
Loading slices from: RONZO_F
Loaded RONZO_F: (224, 224, 111), (111,)
120 ROSARI_NANDO
Loading slices from: ROSARI_NANDO
Loaded ROSARI_NANDO: (224, 224, 75), (75,)
121 RUSCITO_ELISABETTA
Loading slices from: RUSCITO_ELISABETTA
Loaded RUSCITO_ELISABETTA: (224, 224, 38), (38,)
122 RUSNAC_NINA
Loading slices from: RUSNAC_NINA
Loaded RUSNAC_NINA: (224, 224, 77), (77,)
123 RUSSO_IDA
Loading slices from: RUSSO_IDA
Loaded RUSSO_IDA: (224, 224, 73), (73,)
124 SALA_CLARA
Loading slices from: SALA_CLARA
Loaded SALA_CLARA: (224, 224, 70), (70,)
125 SALTARELLI_DOMENICO
Loading slices from: SALTARELLI_DOMENICO
Loaded SALTARELLI_DOMENICO: (224, 224, 75), (75,)
126 SANTINI_ERMANNO
Loading slices from: SANTINI_ERMANNO
Loaded SANTINI_ERMANNO: (224, 224

Loaded FABIANI_ANNA: (224, 224, 64), (64,)
51 FARELLA_PAOLO
Loading slices from: FARELLA_PAOLO
Loaded FARELLA_PAOLO: (224, 224, 32), (32,)
52 FEDERICO_FRANCESCO
Loading slices from: FEDERICO_FRANCESCO
Loaded FEDERICO_FRANCESCO: (224, 224, 56), (56,)
53 FERRAZZA_RITA
Loading slices from: FERRAZZA_RITA
Loaded FERRAZZA_RITA: (224, 224, 59), (59,)
54 FERRI_M_B
Loading slices from: FERRI_M_B
Loaded FERRI_M_B: (224, 224, 56), (56,)
55 FILIPPONI_QUINTINO
Loading slices from: FILIPPONI_QUINTINO
Loaded FILIPPONI_QUINTINO: (224, 224, 66), (66,)
56 FIUCCI_A
Loading slices from: FIUCCI_A
Loaded FIUCCI_A: (224, 224, 168), (168,)
57 FLORIO_FRANCESCO_PAOLO
Loading slices from: FLORIO_FRANCESCO_PAOLO
Loaded FLORIO_FRANCESCO_PAOLO: (224, 224, 65), (65,)
58 FRATINI_RITA
Loading slices from: FRATINI_RITA
Loaded FRATINI_RITA: (224, 224, 58), (58,)
59 GATTAMORTA_NATALINA
Loading slices from: GATTAMORTA_NATALINA
Loaded GATTAMORTA_NATALINA: (224, 224, 70), (70,)
60 GEGGI_GIULIO
Loading slices from: GEGGI_GIU

Loaded STEFANINI_CLORINDA: (224, 224, 64), (64,)
136 STERPA_GIUSEPPE
Loading slices from: STERPA_GIUSEPPE
Loaded STERPA_GIUSEPPE: (224, 224, 82), (82,)
137 SYKULA_GRAZYNA_BARBARA
Loading slices from: SYKULA_GRAZYNA_BARBARA
Loaded SYKULA_GRAZYNA_BARBARA: (224, 224, 69), (69,)
138 TAVERNESE_G
Loading slices from: TAVERNESE_G
Loaded TAVERNESE_G: (224, 224, 120), (120,)
139 TAVOLUCCI_MARIA_RITA
Loading slices from: TAVOLUCCI_MARIA_RITA
Loaded TAVOLUCCI_MARIA_RITA: (224, 224, 76), (76,)
140 TEMPESTINI_MARISA
Loading slices from: TEMPESTINI_MARISA
Loaded TEMPESTINI_MARISA: (224, 224, 63), (63,)
141 TEMPORIN_PATRIZIA
Loading slices from: TEMPORIN_PATRIZIA
Loaded TEMPORIN_PATRIZIA: (224, 224, 70), (70,)
142 TEOFILI_STEFANO
Loading slices from: TEOFILI_STEFANO
Loaded TEOFILI_STEFANO: (224, 224, 70), (70,)
143 TESTA_SERGIO
Loading slices from: TESTA_SERGIO
Loaded TESTA_SERGIO: (224, 224, 59), (59,)
144 TIBERI_GIUSEPPE
Loading slices from: TIBERI_GIUSEPPE
Loaded TIBERI_GIUSEPPE: (224, 224, 91), (

Loaded GIACCHERINI_M: (224, 224, 104), (104,)
64 GIANFELICI_LUISA
Loading slices from: GIANFELICI_LUISA
Loaded GIANFELICI_LUISA: (224, 224, 48), (48,)
65 GIOIA_COSMO_DAMIANO
Loading slices from: GIOIA_COSMO_DAMIANO
Loaded GIOIA_COSMO_DAMIANO: (224, 224, 71), (71,)
66 GIORDANO_STEFANIA
Loading slices from: GIORDANO_STEFANIA
Loaded GIORDANO_STEFANIA: (224, 224, 73), (73,)
67 INCITI_DONATA
Loading slices from: INCITI_DONATA
Loaded INCITI_DONATA: (224, 224, 68), (68,)
68 IONTA_LUCIANA
Loading slices from: IONTA_LUCIANA
Loaded IONTA_LUCIANA: (224, 224, 38), (38,)
69 ISMAIL_A
Loading slices from: ISMAIL_A
Loaded ISMAIL_A: (224, 224, 109), (109,)
70 ISONI_FRANCESCO
Loading slices from: ISONI_FRANCESCO
Loaded ISONI_FRANCESCO: (224, 224, 68), (68,)
71 LABELLA_ADRIANA
Loading slices from: LABELLA_ADRIANA
Loaded LABELLA_ADRIANA: (224, 224, 93), (93,)
72 LANDONE_ANNUNZIATA
Loading slices from: LANDONE_ANNUNZIATA
Loaded LANDONE_ANNUNZIATA: (224, 224, 83), (83,)
73 LIBERATI_G_L
Loading slices from: 

Loaded TOMAO_ANGELO: (224, 224, 72), (72,)
148 TOMEO_VINCENZO
Loading slices from: TOMEO_VINCENZO
Loaded TOMEO_VINCENZO: (224, 224, 43), (43,)
149 TROSCIA_M
Loading slices from: TROSCIA_M
Loaded TROSCIA_M: (224, 224, 75), (75,)
150 VERONESI_ROCCO
Loading slices from: VERONESI_ROCCO
Loaded VERONESI_ROCCO: (224, 224, 25), (25,)
151 VESCIA_M_T
Loading slices from: VESCIA_M_T
Loaded VESCIA_M_T: (224, 224, 87), (87,)
152 VITULANO_RITA
Loading slices from: VITULANO_RITA
Loaded VITULANO_RITA: (224, 224, 63), (63,)
153 ZAMPELLA_A
Loading slices from: ZAMPELLA_A
Loaded ZAMPELLA_A: (224, 224, 136), (136,)
154 ZANATTA_CARLO
Loading slices from: ZANATTA_CARLO
Loaded ZANATTA_CARLO: (224, 224, 76), (76,)
155 ZANGARI_ALDO
Loading slices from: ZANGARI_ALDO
Loaded ZANGARI_ALDO: (224, 224, 74), (74,)
156 ZEPPA_ONORIO
Loading slices from: ZEPPA_ONORIO
Loaded ZEPPA_ONORIO: (224, 224, 78), (78,)
0 ALESSANDRINI_GLAUCO
Loading slices from: ALESSANDRINI_GLAUCO
Loaded ALESSANDRINI_GLAUCO: (224, 224, 63), (63,)

Loaded PRINCIPI_ANNA_MARIA: (224, 224, 72), (72,)
76 PROIETTI_GIOVANNI
Loading slices from: PROIETTI_GIOVANNI
Loaded PROIETTI_GIOVANNI: (224, 224, 51), (51,)
77 PROIETTI_MARIA
Loading slices from: PROIETTI_MARIA
Loaded PROIETTI_MARIA: (224, 224, 90), (90,)
78 QUACQUARELLI_A
Loading slices from: QUACQUARELLI_A
Loaded QUACQUARELLI_A: (224, 224, 57), (57,)
79 QUATTROCIOCCHI_EVELINA
Loading slices from: QUATTROCIOCCHI_EVELINA
Loaded QUATTROCIOCCHI_EVELINA: (224, 224, 65), (65,)
80 RICCI_ALESSANDRO
Loading slices from: RICCI_ALESSANDRO
Loaded RICCI_ALESSANDRO: (224, 224, 49), (49,)
81 ROMITO_ORAZIO
Loading slices from: ROMITO_ORAZIO
Loaded ROMITO_ORAZIO: (224, 224, 53), (53,)
82 RONZO_F
Loading slices from: RONZO_F
Loaded RONZO_F: (224, 224, 111), (111,)
83 ROSARI_NANDO
Loading slices from: ROSARI_NANDO
Loaded ROSARI_NANDO: (224, 224, 75), (75,)
84 RUSCITO_ELISABETTA
Loading slices from: RUSCITO_ELISABETTA
Loaded RUSCITO_ELISABETTA: (224, 224, 38), (38,)
85 RUSNAC_NINA
Loading slices from: 

In [6]:
X = dataset.load_dataset(['T1'], 224, 100)

0 ALESSANDRINI_GLAUCO
Loading slices from: ALESSANDRINI_GLAUCO
Loaded ALESSANDRINI_GLAUCO: (224, 224, 63), (63,)
1 ANGELONI_GIUSEPPINA
Loading slices from: ANGELONI_GIUSEPPINA
Loaded ANGELONI_GIUSEPPINA: (224, 224, 32), (16,)
2 ASSANTO_MARIA
Loading slices from: ASSANTO_MARIA
Loaded ASSANTO_MARIA: (224, 224, 92), (92,)
3 BAGNOLI_VINCENZO
Loading slices from: BAGNOLI_VINCENZO
Loaded BAGNOLI_VINCENZO: (224, 224, 92), (92,)
4 BARONTINI_MARIA_GIOVANNA
Loading slices from: BARONTINI_MARIA_GIOVANNA
Loaded BARONTINI_MARIA_GIOVANNA: (224, 224, 56), (56,)
5 BATTISTA_DOMENICA
Loading slices from: BATTISTA_DOMENICA
Loaded BATTISTA_DOMENICA: (224, 224, 54), (54,)
6 BERGNACH_SILVANO
Loading slices from: BERGNACH_SILVANO
Loaded BERGNACH_SILVANO: (224, 224, 68), (68,)
7 BERNOLA_TERESA
Loading slices from: BERNOLA_TERESA
Loaded BERNOLA_TERESA: (224, 224, 32), (32,)
8 BERTUZZI_LUISA
Loading slices from: BERTUZZI_LUISA
Loaded BERTUZZI_LUISA: (224, 224, 89), (89,)
9 BEVILACQUA_RITA
Loading slices from: B

Loaded NERVEGNA_G: (224, 224, 148), (148,)
83 ORLANDI_PAOLO
Loading slices from: ORLANDI_PAOLO
Loaded ORLANDI_PAOLO: (224, 224, 6), (6,)
84 PAGANNONE_GIANNI
Loading slices from: PAGANNONE_GIANNI
Loaded PAGANNONE_GIANNI: (224, 224, 67), (67,)
85 PAGLIAROLI_LUCIA
Loading slices from: PAGLIAROLI_LUCIA
Loaded PAGLIAROLI_LUCIA: (224, 224, 39), (39,)
86 PAGNOTTA
Loading slices from: PAGNOTTA
Loaded PAGNOTTA: (224, 224, 59), (59,)
87 PALMA
Loading slices from: PALMA
Loaded PALMA: (224, 224, 73), (73,)
88 PALMIERI
Loading slices from: PALMIERI
Loaded PALMIERI: (224, 224, 83), (83,)
89 PANETTI
Loading slices from: PANETTI
Loaded PANETTI: (224, 224, 90), (90,)
90 PASCAL
Loading slices from: PASCAL
Loaded PASCAL: (224, 224, 33), (33,)
91 PASSARI
Loading slices from: PASSARI
Loaded PASSARI: (224, 224, 68), (68,)
92 PIERI
Loading slices from: PIERI
Loaded PIERI: (224, 224, 52), (52,)
93 PIERINI_CATERINA
Loading slices from: PIERINI_CATERINA
Loaded PIERINI_CATERINA: (224, 224, 87), (87,)
94 PINEDA_M

/tf/local/EMANUELA/notebooks/gliomi/dataset.py:215: RuntimeWarning: invalid value encountered in true_divide
  images_centered = (images - u_extended) / s_extended


Loaded RICCI_ALESSANDRO: (224, 224, 49), (49,)
106 ROMITO_ORAZIO
Loading slices from: ROMITO_ORAZIO
Loaded ROMITO_ORAZIO: (224, 224, 53), (53,)
107 ROSARI_NANDO
Loading slices from: ROSARI_NANDO
Loaded ROSARI_NANDO: (224, 224, 75), (75,)
108 RUSCITO_ELISABETTA
Loading slices from: RUSCITO_ELISABETTA
Loaded RUSCITO_ELISABETTA: (224, 224, 38), (38,)
109 RUSNAC_NINA
Loading slices from: RUSNAC_NINA
Loaded RUSNAC_NINA: (224, 224, 77), (77,)
110 RUSSO_IDA
Loading slices from: RUSSO_IDA
Loaded RUSSO_IDA: (224, 224, 73), (73,)
111 SALA_CLARA
Loading slices from: SALA_CLARA
Loaded SALA_CLARA: (224, 224, 70), (70,)
112 SALTARELLI_DOMENICO
Loading slices from: SALTARELLI_DOMENICO
Loaded SALTARELLI_DOMENICO: (224, 224, 75), (75,)
113 SANTINI_ERMANNO
Loading slices from: SANTINI_ERMANNO
Loaded SANTINI_ERMANNO: (224, 224, 43), (43,)
114 SCARAMUZZA_F
Loading slices from: SCARAMUZZA_F
Loaded SCARAMUZZA_F: (224, 224, 104), (104,)
115 SOLOVIY_VOLODYMYR
Loading slices from: SOLOVIY_VOLODYMYR
Loaded SOLO

In [7]:
side = 224

percentile = 100

with open(f"/data/RMN/dataset-gliomi-cnn/datasets-tumor-crop/{sequence_name}-{side}-{percentile}.pickle", "wb") as file:
    pickle.dump(X, file)

In [ ]:
for sequence_name in ["T1", "T2", "ADC", "FLAIR", "MPRAGE", "rCBV"]:
    
    X = dataset.load_dataset([sequence_name], 224, 100, full_brain=True)
    
    sequences = "-".join(sequence_names).lower()
    
    with open(f"/data/RMN/dataset-gliomi-cnn/datasets-full-brain/{sequences}-{side}-{percentile}.pickle", "wb") as file:
        pickle.dump(X, file)

# Generate Datasets

In [ ]:
%%time

root_path = "/data/RMN/dataset-gliomi-cnn"

for dataset in [f"{root_path}/slices-tumor-crop"]:

    for sequence_name in ["T1", "T2", "rCBV", "ADC", "FLAIR", "MPRAGE"]:

        for percentile in [100]:

            print(dataset, sequence_name, percentile)
            
            save_dataset(dataset, [sequence_name], 224, percentile, "/data/RMN/dataset-gliomi-cnn/2-datasets-tumor-crop")        

In [ ]:
%%time

root_path = "/data/RMN/dataset-gliomi-cnn"

for dataset in [f"{root_path}/slices-full-brain"]:

    for sequence_name in ["T1", "T2", "rCBV", "ADC", "FLAIR", "MPRAGE"]:

        for percentile in [100]:

            print(dataset, sequence_name, percentile)
            
            save_dataset(dataset, [sequence_name], 224, percentile, "/data/RMN/dataset-gliomi-cnn/2-datasets-full-brain")        

In [ ]:
X, y = get_dataset_for_classification(f"{root_path}/2-datasets-tumor-crop/t1-224-100-perc.pickle", 
                                      f"{root_path}/dataset-survivor.csv")

In [ ]:
for dataframe in [f"{root_path}/dataset-survivor.csv", 
                  f"{root_path}/dataset-mgmt.csv", 
                  f"{root_path}/dataset-idh.csv", 
                  f"{root_path}/dataset-ki67.csv", 
                  f"{root_path}/dataset-egfr.csv"]:



In [ ]:
%%time

root_path = "/data/RMN/dataset-gliomi-cnn"

for dataframe in [f"{root_path}/dataset-survivor.csv", 
                  f"{root_path}/dataset-mgmt.csv", 
                  f"{root_path}/dataset-idh.csv", 
                  f"{root_path}/dataset-ki67.csv", 
                  f"{root_path}/dataset-egfr.csv"]:

    for dataset in [f"{root_path}/slices-full-brain"]:
        
            for sequence_name in ["T1", "T2", "rCBV", "ADC", "FLAIR", "MPRAGE"]:
                
                for percentile in [100]:
                    
                    print(dataset, dataframe, sequence_name, percentile)
                    save_dataset(dataset, dataframe, [sequence_name], 224, percentile, "/data/RMN/dataset-gliomi-cnn/2-datasets-full-brain")
        